<a href="https://colab.research.google.com/github/yiyc-kr/llm-study/blob/main/llama-3-sqlcoder-8b_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install accelerate # 처음 시작시 그냥 얘만 설치해주고 세션 다시시작하자 무조건 필수!!!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# 시작하기전에 맨 밑에 accelerate, Seq2SeqTrainingArguments 관련된것부터 해결하고 진행하자. 괜히 컴퓨터 단위 날리지말자...

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive_path = 'drive/MyDrive/llm-study/'

In [3]:
# GPU 사용 체크
# TPU를 사용하는 경우 !nvidia-smi 명령어는 작동하지 않습니다.
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jun  1 11:24:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
# 고용량 메모리 사용 체크
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


모델 저장

In [ ]:
!pip install accelerate

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
torch.cuda.is_available()

True

In [8]:
available_memory = torch.cuda.get_device_properties(0).total_memory
available_memory

42481811456

In [9]:
available_memory > 15e9

True

In [2]:
model_name = "defog/llama-3-sqlcoder-8b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# available_memory > 15e9 == True
# if you have atleast 15GB of GPU memory, run load the model in float16
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True,
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

In [7]:
# 모델 저장하기
save_directory = drive_path + "llama-3-sqlcoder-8b"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [8]:
# 로컬 디렉토리에서 모델과 토크나이저 로드
load_directory = drive_path + "llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(load_directory)
model = AutoModelForCausalLM.from_pretrained(load_directory)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

모델 테스트

In [13]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [23]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE corporate_customers (
    corporate_id INTEGER PRIMARY KEY, -- Unique ID for each corporate customer
    name VARCHAR(100) NOT NULL, -- Name of the corporate customer
    address VARCHAR(200), -- Mailing address of the corporate customer
    contact_person VARCHAR(100), -- Contact person in the corporate customer
    contact_phone VARCHAR(20), -- Contact phone number
    contact_email VARCHAR(100) -- Contact email address
);

CREATE TABLE branches (
    branch_id INTEGER PRIMARY KEY, -- Unique ID for each branch
    name VARCHAR(100) NOT NULL, -- Name of the branch
    address VARCHAR(200), -- Address of the branch
    phone VARCHAR(20), -- Contact phone number
    manager VARCHAR(100) -- Branch manager name
);

CREATE TABLE accounts (
    account_id INTEGER PRIMARY KEY, -- Unique ID for each account
    customer_id INTEGER, -- ID of the corporate customer owning the account
    branch_id INTEGER, -- ID of the branch where the account is held
    account_type VARCHAR(50), -- Type of the account (e.g., savings, checking)
    balance DECIMAL(15, 2), -- Current balance of the account
    opened_date DATE, -- Date when the account was opened
    FOREIGN KEY (customer_id) REFERENCES corporate_customers(corporate_id),
    FOREIGN KEY (branch_id) REFERENCES branches(branch_id)
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY, -- Unique ID for each product
    name VARCHAR(100) NOT NULL, -- Name of the product
    description TEXT, -- Description of the product
    interest_rate DECIMAL(5, 2), -- Interest rate for the product
    created_date DATE -- Date when the product was created
);

CREATE TABLE customer_revenues (
    revenue_id INTEGER PRIMARY KEY, -- Unique ID for each revenue record
    customer_id INTEGER, -- ID of the corporate customer
    year INTEGER, -- Year of the revenue record
    total_revenue DECIMAL(15, 2), -- Total revenue for the year
    FOREIGN KEY (customer_id) REFERENCES corporate_customers(corporate_id)
);

CREATE TABLE account_transactions (
    transaction_id INTEGER PRIMARY KEY, -- Unique ID for each transaction
    account_id INTEGER, -- ID of the account involved in the transaction
    transaction_date DATE, -- Date of the transaction
    transaction_type VARCHAR(50), -- Type of the transaction (e.g., deposit, withdrawal)
    amount DECIMAL(15, 2), -- Amount of the transaction
    description TEXT, -- Description or memo for the transaction
    FOREIGN KEY (account_id) REFERENCES accounts(account_id)
);

-- corporate_customers.corporate_id can be joined with accounts.customer_id
-- branches.branch_id can be joined with accounts.branch_id
-- accounts.account_id can be joined with account_transactions.account_id
-- corporate_customers.corporate_id can be joined with customer_revenues.customer_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [10]:
import sqlparse

In [19]:
def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1, # 성능에 따라 조절 1~4 4가 베스트긴 한데 메모리 이슈
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [20]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)
print(generated_sql)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



SELECT p.product_id,
       p.name,
       SUM(s.quantity * p.price) AS total_revenue
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id,
         p.name
ORDER BY total_revenue DESC NULLS LAST;

assistant I am a bot,
and I am here to help you with any questions
or concerns you may have.


In [26]:
question = "Customer names and transaction volume with the most transactions by branch name per month"
generated_sql = generate_query(question)
print(generated_sql)

WITH monthly_transactions AS
  (SELECT a.branch_id,
          a.customer_id,
          COUNT(at.transaction_id) AS num_transactions,
          DATE_TRUNC('month', at.transaction_date) AS MONTH
   FROM accounts a
   JOIN account_transactions AT ON a.account_id = at.account_id
   GROUP BY a.branch_id,
            a.customer_id,
            DATE_TRUNC('month', at.transaction_date)),
     monthly_revenue AS
  (SELECT cr.customer_id,
          SUM(cr.total_revenue) AS total_revenue,
          DATE_TRUNC('month', cr.year) AS MONTH
   FROM customer_revenues cr
   GROUP BY cr.customer_id,
            DATE_TRUNC('month', cr.year'))
SELECT b.name AS branch_name,
       c.name AS customer_name,
       mt.num_transactions,
       mr.total_revenue
FROM monthly_transactions mt
JOIN branches b ON mt.branch_id = b.branch_id
JOIN monthly_revenue mr ON mt.customer_id = mr.customer_id
JOIN corporate_customers c ON mt.customer_id = c.corporate_id
ORDER BY mt.num_transactions DESC,
         mr.total_revenu

In [21]:
load_directory

'drive/MyDrive/llm-study/llama-3-sqlcoder-8b'

In [22]:
# 모델 압축
!tar -cvf drive/MyDrive/llm-study/llama-3-sqlcoder-8b.tar.gz drive/MyDrive/llm-study/llama-3-sqlcoder-8b/

drive/MyDrive/llm-study/llama-3-sqlcoder-8b/
drive/MyDrive/llm-study/llama-3-sqlcoder-8b/tokenizer_config.json
tar: drive/MyDrive/llm-study/llama-3-sqlcoder-8b/tokenizer_config.json: file changed as we read it
drive/MyDrive/llm-study/llama-3-sqlcoder-8b/special_tokens_map.json
tar: drive/MyDrive/llm-study/llama-3-sqlcoder-8b/special_tokens_map.json: file changed as we read it
drive/MyDrive/llm-study/llama-3-sqlcoder-8b/tokenizer.json
drive/MyDrive/llm-study/llama-3-sqlcoder-8b/config.json
tar: drive/MyDrive/llm-study/llama-3-sqlcoder-8b/config.json: file changed as we read it
drive/MyDrive/llm-study/llama-3-sqlcoder-8b/generation_config.json
tar: drive/MyDrive/llm-study/llama-3-sqlcoder-8b/generation_config.json: file changed as we read it
drive/MyDrive/llm-study/llama-3-sqlcoder-8b/model-00001-of-00004.safetensors
drive/MyDrive/llm-study/llama-3-sqlcoder-8b/model-00002-of-00004.safetensors
tar: drive/MyDrive/llm-study/llama-3-sqlcoder-8b/model-00002-of-00004.safetensors: file changed 

8bit 모델을 다운받자

In [3]:
# GPU 사용 체크
# TPU를 사용하는 경우 !nvidia-smi 명령어는 작동하지 않습니다.
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun  3 06:39:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install accelerate # 처음 시작시 그냥 얘만 설치해주고 세션 다시시작하자 무조건 필수!!!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
drive_path = 'drive/MyDrive/llm-study/'

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
torch.__version__

'2.3.0+cu121'

In [5]:
model_name = "defog/llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 8 bits
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
# 모델 저장하기
save_directory = drive_path + "llama-3-sqlcoder-8b-8bits"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)